In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 15 16:39:32 2020

@author: erdem
"""

# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT  ?businessLabel ?headquarters_locationLabel ?countryLabel ?employees WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?business wdt:P31 wd:Q4830453.
  OPTIONAL {  }
  OPTIONAL { ?business wdt:P159 ?headquarters_location. }
  OPTIONAL { ?business wdt:P17 ?country. }
  OPTIONAL { ?business wdt:P1128 ?employees. }
}
LIMIT 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


In [2]:
df = get_sparql_dataframe(endpoint_url, query)



In [3]:
df

,businessLabel,headquarters_locationLabel,countryLabel,employees
0,Bertone,Turin,Italy,1360
1,Bitter Automobile,Ennepetal,Germany,None
2,Borgward,Bremen,Germany,None
3,Bristol Cars,Filton,United Kingdom,None
4,Automobiles Ettore Bugatti,Molsheim,France,None
5,Buick,Detroit,United States of America,None
6,BYD Auto,Shenzhen,None,60000
7,Bethlehem Steel,Bethlehem,None,None
8,Dacia,Mioveni,None,12532
9,New Directions Publishing,New York City,United States of America,None
